In [2]:
import pandas as pd

addrs = pd.read_csv('addr_info.csv', index_col=0)
addrs = addrs[~((addrs.duplicated(subset=['address'])) & 
          (~addrs['name'].isin(['binance', 'coinbase', 'bullish'])))].drop_duplicates(subset=['address'])
addrs.name = addrs.name.str.lower()
feats = ['name', 'serviceType', 'sub1Service', 'walletPurpose', 'sub1Wallet']
addrs.columns
# addrs[addrs.address.isin(addrs[addrs.duplicated(subset=['address'])].address)]

Index(['address', 'name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet'],
      dtype='object')

In [2]:
addrs.loc[addrs.name.isin(['binance', 'coinbase', 'bullish'])].value_counts(['name', 'serviceType', 'sub1Service', 'walletPurpose', 'sub1Wallet'])

name      serviceType  sub1Service  walletPurpose  sub1Wallet
binance   CeFi         exc          Vault          deposit       11341
coinbase  CeFi         exc          Vault          deposit        6188
binance   CeFi         exc          Vault          hot              24
                       fund         Vault          deposit          20
bullish   CeFi         exc          Vault          deposit          11
coinbase  CeFi         exc          Vault          hot               8
                       fund         Vault          deposit           7
                       exc          Vault          cold              6
binance   CeFi         custody      Vault          deposit           1
bullish   CeFi         exc          Vault          hot               1
coinbase  CeFi         exc          Vault          proxy             1
          DeFi         exc          Vault          proxy             1
Name: count, dtype: int64

In [3]:
import duckdb

try:
    conn = duckdb.connect('duckdb_test.db', read_only=False)
    print('DuckDB connection established successfully')
except Exception as e:
    print(f'Error connecting to DuckDB: {str(e)}')
    exit(1)

DuckDB connection established successfully


# ETH Network


In [4]:
tokens = dict(
    tether='0xdAC17F958D2ee523a2206206994597C13D831ec7',
    usdc='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    dai='0x6B175474E89094C44Da98b954EedeAC495271d0F',
    fdusd='0xc5f0f7b66764F6ec8C8Dff7BA683102295E16409',
    weth='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
    wbtc='0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
    bnb='0xB8c77482e45F1F44dE1745F52C74426C631bDD52',
    ada='0x3ee2200efb3400fabb9aacf31297cbdd1d435d47',
    link='0x514910771af9ca656af840dff83e8264ecf986ca',
    aethusdt='0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a',
    cusdt='0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9'

)
tokens = {k: v.lower() for k, v in tokens.items()}


### Grap Addresses on each month


In [ ]:
for year in range(2024,2025):
    for month in range(7, 11):
        # Create the table
        df_martial_law = conn.execute(f"""
            SELECT *
            FROM read_parquet('ethereum_token_transfer/*/*/*.parquet', hive_partitioning = true)
            WHERE year={year} AND month={month}
        """).df()

        for k, v in tokens.items():
            df_martial_law.loc[df_martial_law.token_address==v, 'token_name'] = k
            
        for v in ['token', 'from', 'to']:
            df_martial_law = df_martial_law.merge(
            addrs[['address']+feats], 
            left_on=f'{v}_address', right_on='address', 
            how="left", validate='m:1'
        ).rename(columns={c:f'{v}_{c}' for c in feats}).drop(columns=['address'])
        df_martial_law['token_name'] = df_martial_law.filter(like='token_name').bfill(axis=1).iloc[:, 0]
        df_martial_law = df_martial_law.loc[:, ~df_martial_law.columns.duplicated()]

        df_ml_graph = df_martial_law[[
            'from_name', 'to_name', 'transaction_hash', 'raw_amount', 'block_timestamp', 'from_address', 'to_address'
        ]].copy().reset_index(drop=True)
        for v in ['from', 'to']:
            df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_name'] =  df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_address' ]
        

        df_ml_graph = df_ml_graph.drop(columns=['from_address', 'to_address'])
        sims = pd.concat([
            df_ml_graph.from_name.value_counts().reset_index().rename(columns={'from_name': 'name'}),
            df_ml_graph.to_name.value_counts().reset_index().rename(columns={'to_name': 'name'})])
        simss = sims.groupby('name').sum('count')
        simss['year'] = year
        simss['month'] = month
        simss.to_csv(f'meta/{year}_{month}.csv')

### Save Labelled Dataset


In [5]:
target_tokens = [
    'tether', 'usdc', #'tethertreasury', 'circle'
]

# 603 minutes

# for year in range(2016,2025):
# for year in [2025]:
    # for month in range(1, 6):/
# for year, month in [(2025, 5), (2015, 12), (2015, 11), (2015, 10)]:
for year, month in [(2025, i) for i in range(1, 4)] + [(2024, i) for i in range(7, 11)]:
        # Create the table
        df_martial_law = conn.execute(f"""
            SELECT *
            FROM read_parquet('ethereum_token_transfer/*/*/*.parquet', hive_partitioning = true)
            WHERE year={year} AND month={month}
        """).df()

        for k, v in tokens.items():
            if k not in target_tokens:
                continue
            df_martial_law.loc[df_martial_law.token_address==v, 'token_name'] = k
            
        for v in ['token', 'from', 'to']:
            df_martial_law = df_martial_law.merge(
            addrs[['address']+feats], 
            left_on=f'{v}_address', right_on='address', 
            how="left", validate='m:1'
        ).rename(columns={c:f'{v}_{c}' for c in feats}).drop(columns=['address'])
        df_martial_law['token_name'] = df_martial_law.filter(like='token_name').bfill(axis=1).iloc[:, 0]
        df_martial_law = df_martial_law.loc[:, ~df_martial_law.columns.duplicated()]
        df_ml = df_martial_law.reset_index(drop=True)
        # for v in ['from', 'to']:
        #     df_ml.loc[df_ml[f'{v}_name'].isna(), f'{v}_name'] =  df_ml.loc[df_ml[f'{v}_name'].isna(), f'{v}_address' ]
    
        df_ml = df_ml.loc[~(df_ml.from_name.isna() & df_ml.to_name.isna())]
        df_ml.loc[df_ml.from_name.isna(), 'from_name'] = 'idk'
        df_ml.loc[df_ml.to_name.isna(), 'to_name'] = 'idk'
        df_ml = df_ml.drop(columns=['from_address', 'to_address', 'token_address'])
        df_ml.loc[df_ml.token_name.isin(target_tokens)].to_parquet(
            f'labelled/eth_martial_law_{year}_{month:02d}.parquet',
        )


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# TRON Newtork


In [4]:
import os
import pandas as pd
os.makedirs('meta_tron', exist_ok=True)

tokens = dict(
    tether='TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t',
    # studst='TThzxNRLrW2Brp9DcTQU8i4Wd9udCWEdZ3',
    # usdc='TEkxiTehnzSmSe2XqrBj4w32RUN966rdz8',
    # weth='T9d1b7c8e2f3c4d5e6f7a8b9c0d1e2f3g4h5i6j7n',
)
tokens = {k: v.lower() for k, v in tokens.items()}

### Grep Addresses on each month


In [ ]:


for year in [2024]: # range(2024,2026):
    for month in range(10, 13):
        # Create the table
        df_martial_law = conn.execute(f"""
            SELECT *
            FROM read_parquet('tron_token_transfer/*/*/*.parquet', hive_partitioning = true)
            WHERE year={year} AND month={month}
        """).df()
        if df_martial_law.empty:
            print(f"No data for {year}-{month}, skipping...")
            continue

        for k, v in tokens.items():
            df_martial_law.loc[df_martial_law.token_address==v, 'token_name'] = k
            
        for v in ['token', 'from', 'to']:
            df_martial_law = df_martial_law.merge(
            addrs[['address']+feats], 
            left_on=f'{v}_address', right_on='address', 
            how="left", validate='m:1'
        ).rename(columns={c:f'{v}_{c}' for c in feats}).drop(columns=['address'])
        df_martial_law['token_name'] = df_martial_law.filter(like='token_name').bfill(axis=1).iloc[:, 0]
        df_martial_law = df_martial_law.loc[:, ~df_martial_law.columns.duplicated()]



        df_ml_graph = df_martial_law[[
            'from_name', 'to_name', 'transaction_hash', 'raw_amount', 'block_timestamp', 'from_address', 'to_address'
        ]].copy().reset_index(drop=True)
        for v in ['from', 'to']:
            df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_name'] =  df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_address' ]
        

        df_ml_graph = df_ml_graph.drop(columns=['from_address', 'to_address'])
        sims = pd.concat([
            df_ml_graph.from_name.value_counts().reset_index().rename(columns={'from_name': 'name'}),
            df_ml_graph.to_name.value_counts().reset_index().rename(columns={'to_name': 'name'})])
        simss = sims.groupby('name').sum('count')
        simss['year'] = year
        simss['month'] = month
        simss.to_csv(f'meta_tron/{year}_{month}.csv')

### Save Labelled Dataset


In [5]:

# for year, month in [(y, m) for y in range(2020, 2025) for m in range(1, 13)] + [
#     (2025, i) for i in range(1, 6)] + [(2019, i) for i in range(7, 13)]:
for year, month in 
# /[(2023, i) for i in range(5, 7)] + [(2024, i) for i in range(11, 13)]:
        try:
            df = conn.execute(f"""
                SELECT *
                FROM read_parquet('tron_token_transfer/*/*/*.parquet', hive_partitioning = true)
                WHERE year={year} AND month={month}
            """).df()
        except Exception as e:
            print(f"Error reading data for {year}-{month}: {e}")
            continue
        
        df.from_address = df.from_address.str.lower()
        df.to_address = df.to_address.str.lower()
        df.token_address = df.token_address.str.lower()
        
        df = df.loc[df.token_address==(tokens['tether'])].reset_index(drop=True)
        df['token_name']='tether'
        # for k, v in tokens.items():
        #     df.loc[df.token_address==v, 'token_name'] = k
            
        for v in ['to', 'from']:
            df = df.merge(
            addrs[['address']+feats], 
            left_on=f'{v}_address', right_on='address', 
            how="left", validate='m:1'
        ).rename(columns={c:f'{v}_{c}' for c in feats}).drop(columns=['address'])
            
        df['token_name'] = df.filter(like='token_name').bfill(axis=1).iloc[:, 0]
        df = df.loc[:, ~df.columns.duplicated()].drop(columns=['from_address', 'to_address', 'token_address'])

        df_ml = df.reset_index(drop=True)
        df_ml = df_ml.loc[~(df_ml.from_name.isna() & df_ml.to_name.isna())]
        df_ml.loc[df_ml.from_name.isna(), 'from_name'] = 'idk'
        df_ml.loc[df_ml.to_name.isna(), 'to_name'] = 'idk'
        
        df_ml = df_ml
        df_ml.to_parquet(
            f'labelled/tron_martial_law_{year}_{month:02d}.parquet',
        )

Error reading data for 2023-5: Invalid Input Error: No magic bytes found at end of file 'tron_token_transfer\year=2023\month=5\data_0.parquet'
Error reading data for 2023-6: Invalid Input Error: No magic bytes found at end of file 'tron_token_transfer\year=2023\month=6\data_0.parquet'


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Refining


In [1]:
import pyarrow as pa
import pyarrow.compute as pc

import os
import pyarrow.parquet as pq

def replace_long_with_idk(col):
    mask = pc.greater(pc.utf8_length(pc.cast(col, pa.string())), 20)
    return pc.if_else(mask, pa.array(["idk"] * len(col)), col)

transformations = {
    'transaction_hash': lambda col: pc.dictionary_encode(pc.cast(col, pa.string())).combine_chunks().indices
}


parquet_dir = 'labelled'
col_to_replace = ['from_name', 'to_name']

for filename in os.listdir(parquet_dir):
    if not (filename.endswith('.parquet')):
        continue
    if 'tron' not in filename:
        continue
    
    src_file_path = os.path.join(parquet_dir, filename)
    dest_file_path = os.path.join(parquet_dir, filename.split('_')[0], filename)
    # if os.path.exists('filtered_'+dest_file_path):
    #    continue
        
    table = pq.read_table(src_file_path, columns=[
        'block_timestamp', 'raw_amount', 
        'from_name', 'to_name', 'token_name',
        'from_serviceType', 'from_sub1Service', 'from_walletPurpose', 'from_sub1Wallet',
        'to_serviceType', 'to_sub1Service', 'to_walletPurpose', 'to_sub1Wallet',
    ])
    # for col_name in table.column_names:
    #     null_count = pc.count(table[col_name], mode='only_null').as_py()
    #     if null_count > 0:
    #         print(f"Column '{col_name}' has {null_count} null values")
        
    # for c in col_to_replace:
    #     table = table.set_column(table.schema.get_field_index(c), c, replace_long_with_idk(table[c]))

    # filtered_table = table.filter(pc.invert(pc.and_(
    #     pc.equal(table['from_name'], 'idk'),
    #     pc.equal(table['to_name'], 'idk')
    # )))
    
    filtered_table = table
    

    # for col_name, transform_func in transformations.items():
    #     try:
    #         filtered_table = filtered_table.set_column(
    #             filtered_table.schema.get_field_index(col_name),
    #             col_name,
    #             transform_func(filtered_table[col_name])
    #         )
    #     except:
    #         print(f"Error transforming column {col_name} in {filename}")
    #         continue
      
    pa.parquet.write_table(filtered_table, 'filtered_' + dest_file_path)
    print(f"Processed {filename}, saved to filtered_{dest_file_path}")
 
    

Processed tron_martial_law_2025_04.parquet, saved to filtered_labelled\tron\tron_martial_law_2025_04.parquet
Processed tron_martial_law_2025_03.parquet, saved to filtered_labelled\tron\tron_martial_law_2025_03.parquet
Processed tron_martial_law_2025_02.parquet, saved to filtered_labelled\tron\tron_martial_law_2025_02.parquet
Processed tron_martial_law_2025_01.parquet, saved to filtered_labelled\tron\tron_martial_law_2025_01.parquet
Processed tron_martial_law_2024_12.parquet, saved to filtered_labelled\tron\tron_martial_law_2024_12.parquet
Processed tron_martial_law_2024_11.parquet, saved to filtered_labelled\tron\tron_martial_law_2024_11.parquet
Processed tron_martial_law_2020_01.parquet, saved to filtered_labelled\tron\tron_martial_law_2020_01.parquet
Processed tron_martial_law_2020_02.parquet, saved to filtered_labelled\tron\tron_martial_law_2020_02.parquet
Processed tron_martial_law_2020_03.parquet, saved to filtered_labelled\tron\tron_martial_law_2020_03.parquet
Processed tron_mart